In [31]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from keras.layers.core import Dense, Activation
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier

In [32]:
df = pd.read_csv("kddcup.data_10_percent_corrected")

In [33]:
df.columns = [
    'duration',
    'protocol_type',
    'service',
    'flag',
    'src_bytes',
    'dst_bytes',
    'land',
    'wrong_fragment',
    'urgent',
    'hot',
    'num_failed_logins',
    'logged_in',
    'num_compromised',
    'root_shell',
    'su_attempted',
    'num_root',
    'num_file_creations',
    'num_shells',
    'num_access_files',
    'num_outbound_cmds',
    'is_host_login',
    'is_guest_login',
    'count',
    'srv_count',
    'serror_rate',
    'srv_serror_rate',
    'rerror_rate',
    'srv_rerror_rate',
    'same_srv_rate',
    'diff_srv_rate',
    'srv_diff_host_rate',
    'dst_host_count',
    'dst_host_srv_count',
    'dst_host_same_srv_rate',
    'dst_host_diff_srv_rate',
    'dst_host_same_src_port_rate',
    'dst_host_srv_diff_host_rate',
    'dst_host_serror_rate',
    'dst_host_srv_serror_rate',
    'dst_host_rerror_rate',
    'dst_host_srv_rerror_rate',
    'outcome'
]

#display 5 rows
df[0:5]

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,outcome
0,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,217,2032,0,0,0,0,...,59,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.


In [34]:
#Convert target into False and True 
#df['target'] = df['outcome'] != 'normal'

#Features
x = df.drop(['outcome'], axis = 1)

#Outcomes - Turning into Dummy variables
#y = df['outcome']
dummies = pd.get_dummies(df['outcome'])
y = dummies.values

df = pd.DataFrame(x)

df[0:5]

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,tcp,http,SF,239,486,0,0,0,0,...,19,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0
1,0,tcp,http,SF,235,1337,0,0,0,0,...,29,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0
2,0,tcp,http,SF,219,1337,0,0,0,0,...,39,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0
3,0,tcp,http,SF,217,2032,0,0,0,0,...,49,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0
4,0,tcp,http,SF,217,2032,0,0,0,0,...,59,59,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0


In [35]:
#Turn categorical data into intergers of 0 & 1 

dummies = pd.get_dummies(df['protocol_type'],['service'],['flag'],['land'], ['logged_in'],['is_host_login'],['is_guest_login'])

#drop as column is now encoded
x = x.drop(['protocol_type','service','flag','land','logged_in','is_host_login','is_guest_login'], axis = 1)

#concat dummy values to the dataframe

x = x.join(dummies)

x[0:5]

,duration,src_bytes,dst_bytes,wrong_fragment,urgent,hot,num_failed_logins,num_compromised,root_shell,su_attempted,...,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,['service']['flag']tcp,['service']['flag']udp,['service']['flag']nan
0,0,239,486,0,0,0,0,0,0,0,...,0.0,0.05,0.0,0.0,0.0,0.0,0.0,1,0,0
1,0,235,1337,0,0,0,0,0,0,0,...,0.0,0.03,0.0,0.0,0.0,0.0,0.0,1,0,0
2,0,219,1337,0,0,0,0,0,0,0,...,0.0,0.03,0.0,0.0,0.0,0.0,0.0,1,0,0
3,0,217,2032,0,0,0,0,0,0,0,...,0.0,0.02,0.0,0.0,0.0,0.0,0.0,1,0,0
4,0,217,2032,0,0,0,0,0,0,0,...,0.0,0.02,0.0,0.0,0.0,0.0,0.0,1,0,0


In [36]:
# Get column names first
names = x.columns
# Create the Scaler object
scaler = preprocessing.StandardScaler()
# Fit data on the scaler object
scaled_df = scaler.fit_transform(x)
scaled_df = pd.DataFrame(scaled_df, columns=names)

x = pd.DataFrame(scaled_df)

x[0:5]

,duration,src_bytes,dst_bytes,wrong_fragment,urgent,hot,num_failed_logins,num_compromised,root_shell,su_attempted,...,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,['service']['flag']tcp,['service']['flag']udp,['service']['flag']nan
0,-0.067792,-0.002820,-0.011578,-0.04772,-0.002571,-0.044136,-0.009782,-0.005679,-0.010552,-0.004676,...,-0.282867,-1.146740,-0.158629,-0.464418,-0.463203,-0.25204,-0.249464,1.264607,-0.207295,0.0
1,-0.067792,-0.002824,0.014179,-0.04772,-0.002571,-0.044136,-0.009782,-0.005679,-0.010552,-0.004676,...,-0.282867,-1.188293,-0.158629,-0.464418,-0.463203,-0.25204,-0.249464,1.264607,-0.207295,0.0
2,-0.067792,-0.002840,0.014179,-0.04772,-0.002571,-0.044136,-0.009782,-0.005679,-0.010552,-0.004676,...,-0.282867,-1.188293,-0.158629,-0.464418,-0.463203,-0.25204,-0.249464,1.264607,-0.207295,0.0
3,-0.067792,-0.002842,0.035214,-0.04772,-0.002571,-0.044136,-0.009782,-0.005679,-0.010552,-0.004676,...,-0.282867,-1.209070,-0.158629,-0.464418,-0.463203,-0.25204,-0.249464,1.264607,-0.207295,0.0
4,-0.067792,-0.002842,0.035214,-0.04772,-0.002571,-0.044136,-0.009782,-0.005679,-0.010552,-0.004676,...,-0.282867,-1.209070,-0.158629,-0.464418,-0.463203,-0.25204,-0.249464,1.264607,-0.207295,0.0


In [37]:
#Perform PCA to reduce dimensionality of feature vector for better accuracy
#reduce from 37 to 2 dimensional space

pca=PCA(n_components = 2)
pca.fit(x)
x_scaled = pca.transform(x)
print ("Original Shape:   ", x.shape)
print ("Transformed Shape:",x_scaled.shape)

x = x_scaled


Original Shape:    (494020, 37)
Transformed Shape: (494020, 2)


In [38]:
# Create a test/train split, testing on 25% of data. 

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state=42)

In [40]:
# Creating a neural network 

model = Sequential()

model.add(Dense(120, input_dim =x.shape[1], activation='relu', kernel_initializer='normal'))
model.add(Dense(60, input_dim=x.shape[1], activation='relu', kernel_initializer='normal', )) 
model.add(Dense(30, input_dim=x.shape[1],  activation='relu', kernel_initializer='normal',)) 
model.add(Dense(1, kernel_initializer='normal'))
model.add(Dense(y.shape[1],activation='softmax')) #takes in an input and spits out 1D vector

#Add in hyper parameters
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
#Early stopping used to stop model from over or under fitting - stops when accuracy is no longer improving 
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=6, verbose=3, mode='auto')
#Fit the model
model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor],verbose=2,epochs=100)

Train on 370515 samples, validate on 123505 samples
Epoch 1/100
 - 16s - loss: 0.1449 - acc: 0.9644 - val_loss: 0.0678 - val_acc: 0.9841
Epoch 2/100
 - 14s - loss: 0.0643 - acc: 0.9846 - val_loss: 0.0595 - val_acc: 0.9855
Epoch 3/100
 - 16s - loss: 0.0588 - acc: 0.9860 - val_loss: 0.0558 - val_acc: 0.9868
Epoch 4/100
 - 17s - loss: 0.0574 - acc: 0.9863 - val_loss: 0.0575 - val_acc: 0.9868
Epoch 5/100
 - 18s - loss: 0.0563 - acc: 0.9868 - val_loss: 0.0541 - val_acc: 0.9864
Epoch 6/100
 - 17s - loss: 0.0550 - acc: 0.9870 - val_loss: 0.0526 - val_acc: 0.9871
Epoch 7/100
 - 19s - loss: 0.0546 - acc: 0.9872 - val_loss: 0.0511 - val_acc: 0.9878
Epoch 8/100
 - 15s - loss: 0.0540 - acc: 0.9874 - val_loss: 0.0508 - val_acc: 0.9888
Epoch 9/100
 - 14s - loss: 0.0536 - acc: 0.9875 - val_loss: 0.0536 - val_acc: 0.9865
Epoch 10/100
 - 15s - loss: 0.0527 - acc: 0.9878 - val_loss: 0.0509 - val_acc: 0.9878
Epoch 11/100
 - 13s - loss: 0.0525 - acc: 0.9880 - val_loss: 0.0489 - val_acc: 0.9869
Epoch 12/10

In [41]:
# Measure accuracy
predictors = model.predict(x_test)
predictors = np.argmax(predictors,axis=1)
prediction = np.argmax(y_test,axis=1)
score = metrics.accuracy_score(prediction, predictors)
print("Accuracy:", score * 100,"%")


Accuracy: 98.9692724990891 %


In [42]:
from sklearn.metrics import classification_report
print(classification_report(prediction,predictors)) 

              precision    recall  f1-score   support

           0       0.75      0.21      0.33       540
           1       0.00      0.00      0.00        11
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         9
           4       0.00      0.00      0.00         3
           5       0.89      0.80      0.84       325
           6       0.00      0.00      0.00         5
           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         2
           9       1.00      1.00      1.00     26666
          10       0.00      0.00      0.00        50
          11       0.96      1.00      0.98     24285
          12       0.00      0.00      0.00         1
          13       0.00      0.00      0.00         2
          14       0.00      0.00      0.00        50
          15       0.89      0.92      0.90       278
          16       0.00      0.00      0.00         2
          17       0.79    

C:\Users\avmas\Miniconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
